In [ ]:
# Imports
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils.eval import geoguessr_score
from constants import *

# Get rid of pandas max width
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# User defined params
USE_LATEST_RUNS = True # use latest per agent or hand-selected run ("top")
results_dir = ALL_RUNS_DIR if USE_LATEST_RUNS else TOP_RUNS_DIR

# Get runs for each agent
result_filenames = {}
for agent in AGENT_LIST:
    # Get all runs for agent
    runs = [x for x in os.listdir(results_dir) if x.startswith(agent)]
    latest_run = sorted(runs, key=lambda x: os.path.getmtime(os.path.join(results_dir, x)))[-1]
    result_filenames[agent] = latest_run
print(result_filenames)

# Compile results
agg_results = {}
df = pd.DataFrame()
for agent, result_filename in result_filenames.items():
    # Load results
    with open(f"{results_dir}/{result_filename}", "r") as file:
        results = json.load(file)

    # Append dict to df
    new_row = pd.DataFrame({k: v["distance"] for k, v in results.items()}, index=[agent])
    if len(df) == 0:
        df = new_row
    else:
        df = pd.concat([df, new_row])

display(df)

In [ ]:
## Calculate various metrics/features
# Get mean, min distance
t1_cols = [col for col in df.columns if int(col) <= 110]
t2_cols = [col for col in df.columns if int(col) >= 200]
df["mean"] = df[t1_cols + t2_cols].mean(axis=1)
df["mean_t1"] = df[t1_cols].mean(axis=1)
df["mean_t2"] = df[t2_cols].mean(axis=1)
df["min"] = df[t1_cols + t2_cols].min(axis=1)
df = df[sorted(df.columns)]
display(df)

# Get score (by tier)
df["total_score"] = 0
df["total_score_t1"] = 0
df["total_score_t2"] = 0
for index, row in df.iterrows():
    for col in df.columns:
        if col[0] in ["1", "2"]:
            score = geoguessr_score(df.at[index, col])
            df.at[index, "total_score"] += score
            if int(col) <= 110:
                df.at[index, "total_score_t1"] += score
            elif int(col) > 110:
                df.at[index, "total_score_t2"] += score

# Calculate normalized scores by the number of non-NA records for each score column
score_columns = ['total_score', 'total_score_t1', 'total_score_t2']
for index, row in df.iterrows():
    non_na_t1 = df.loc[index, '101':'110'].count()
    non_na_t2 = df.loc[index, '201':'210'].count()
    
    df.at[index, 'normalized_total_score_t1'] = df.at[index, 'total_score_t1'] / non_na_t1 if non_na_t1 > 0 else 0
    df.at[index, 'normalized_total_score_t2'] = df.at[index, 'total_score_t2'] / non_na_t2 if non_na_t2 > 0 else 0
    df.at[index, 'normalized_total_score'] = df.at[index, 'total_score'] / (non_na_t1 + non_na_t2) if (non_na_t1 + non_na_t2) > 0 else 0


## Metrics

## Plot Points

In [ ]:
# Plot score columns for each agent type using a bar chart
plt.figure(figsize=(10, 5))
score_columns = ['total_score', 'total_score_t1', 'total_score_t2']
width = 0.25  # width of the bars

# Create bar positions
x = np.arange(len(df.index))
for i, score in enumerate(score_columns):
    plt.bar(x + i * width, df[score], width=width, label=score)

plt.xlabel('Agent Type')
plt.ylabel('Score')
plt.title('Scores by Agent Type')
plt.legend()
plt.xticks(x + width, df.index, rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Plot normalized score columns for each agent type using a bar chart
plt.figure(figsize=(10, 5))
score_columns = ['normalized_total_score', 'normalized_total_score_t1', 'normalized_total_score_t2']
width = 0.25  # width of the bars

# Create bar positions
x = np.arange(len(df.index))
for i, score in enumerate(score_columns):
    plt.bar(x + i * width, df[score], width=width, label=score)

plt.xlabel('Agent Type')
plt.ylabel('Normalized Score')
plt.title('Normalized Scores by Agent Type (ignoring NAs)')
plt.legend(loc='lower right')
plt.xticks(x + width, df.index, rotation=45)
plt.tight_layout()
plt.show()


### Average Distance (excluding NANs)

In [ ]:
# Plot normalized score columns for each agent type using a bar chart
plt.figure(figsize=(10, 5))
score_columns = ['mean', 'mean_t1', 'mean_t2']
width = 0.25  # width of the bars

# Create bar positions
x = np.arange(len(df.index))
for i, score in enumerate(score_columns):
    plt.bar(x + i * width, df[score], width=width, label=score)

plt.xlabel('Agent Type')
plt.ylabel('Mean Distance')
plt.title('Mean Distance by Agent Type (ignoring NAs)')
plt.legend(loc='upper right')
plt.xticks(x + width, df.index, rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Plot a bar chart of the mean column
plt.figure(figsize=(10, 4))
plt.bar(df.index, df['mean'], color='skyblue')
plt.xlabel('Solution Type')
plt.ylabel('Mean Distance (in km)')
plt.title('Mean Distance (in km) by Agent Type')
plt.xticks(rotation=45)
plt.show()

### Error Plots by Agent Type

#### 100 Level

In [ ]:
# Plotting values for each index for specified columns with multi-bar chart
plt.figure(figsize=(10, 5))
index_labels = df.index
bar_width = 0.2
x = np.arange(len(t1_cols))

for i, index_label in enumerate(index_labels):
    values = df.loc[index_label, t1_cols].fillna(-99)  # Replace NA values with -99
    plt.bar(x + i * bar_width, values, bar_width, label=index_label)

plt.xlabel('Location ID')
plt.ylabel('Error Distance (km)')
plt.title('Error Distance for 100-Level Locations by Agent Type')
plt.xticks(x + bar_width * (len(index_labels) - 1) / 2, t1_cols)
plt.xticks(rotation=45)
plt.ylim(-100, 1000)
plt.legend()
plt.show()


In [ ]:
# Plotting values for each index for specified columns with multi-bar chart
plt.figure(figsize=(10, 5))
index_labels = df.index
bar_width = 0.2
x = np.arange(len(t2_cols))

for i, index_label in enumerate(index_labels):
    values = df.loc[index_label, t2_cols].fillna(-999)  # Replace NA values with -99
    plt.bar(x + i * bar_width, values, bar_width, label=index_label)

plt.xlabel('Location ID')
plt.ylabel('Error Distance (km)')
plt.title('Error Distance for 200-Level Locations by Agent Type')
plt.xticks(x + bar_width * (len(index_labels) - 1) / 2, t2_cols)
plt.xticks(rotation=45)
plt.ylim(-1000, 10000)
plt.legend()
plt.show()

In [ ]:
# Calculate the percentage of NA values for each index in the DataFrame and plot as a bar chart
# Splitting the NA count by total NA rate, NA rate for t1_cols and t2_cols

na_percentage_total = round(df[t1_cols + t2_cols].isna().mean(axis=1) * 100, 2)
na_percentage_t1 = round(df[t1_cols].isna().mean(axis=1) * 100, 2)
na_percentage_t2 = round(df[t2_cols].isna().mean(axis=1) * 100, 2)

plt.figure(figsize=(15, 6))
x = np.arange(len(df.index))
width = 0.25  # width of the bars

plt.bar(x, na_percentage_total, color='red', width=width, label='Total NA Rate')
plt.bar(x + width, na_percentage_t1, color='blue', width=width, label='T1 NA Rate')
plt.bar(x + 2 * width, na_percentage_t2, color='green', width=width, label='T2 NA Rate')

plt.xlabel('Agent Type')
plt.ylabel('Rate of NA Values (%)')
plt.title('Rate of NA Values by Agent Type and Location Tier')
plt.xticks(x + width, df.index, rotation=45)
plt.legend()
plt.show()
